In [6]:
from einops import rearrange, reduce
import numpy as np
from utils import guess

In [7]:
x = np.random.RandomState(42).normal(size=[10, 32, 100, 200])

In [8]:
framework = 'pytorch'

In [10]:
if framework == 'tensorflow':
    import tensorflow as tf 
    tape = tf.GradientTape(persistent=True)
    tape.__enter__()
    x = tf.Variable(x) + 0
elif framework == 'pytorch':
    import torch 
    
    x = torch.from_numpy(x)
    x.require_grads = True


In [12]:
y = rearrange(x, "b c h w -> b h w c")
guess(y.shape)

Answer is: (10, 100, 200, 32) (hover to see)

### Backpropagation

In [18]:
y0 = x
y1 = reduce(y0, "b c h w -> b c", "max")
y2 = rearrange(y1, "b c -> c b")
y3 = reduce(y2, "c b -> ", "sum")

In [ ]:

if framework == "tensorflow":
    print(reduce(tape.gradient(y3, x), "b c h w -> ", "sum"))
else:
    y3.backward()
    print(reduce(x.grad, "b c h w -> ", "sum"))